In [2]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from IPython.display import clear_output
import pickle

#common utilities for transforming data
from commons import Utilities
utils = Utilities()

In [3]:
medians = pickle.load( open( "../medians.p", "rb" ) )
models = pickle.load( open( "../models.p", "rb" ) )

FileNotFoundError: [Errno 2] No such file or directory: '../medians.p'

In [4]:
#Predict and write to file

submissions = []

def process(df_test,firstChunk,models):
    df_test = normalize_data_w_medians(df_test,medians)
    df_test['Id2'] = df_test['Id']
    X_test = df_test.drop(['Id'],axis=1)
    for model in models:
        y_pred = model.predict(X_test)
        submission = pd.DataFrame({
            "Id":df_test['Id'],
            "Response":y_pred
        })
        sub_index=models.index(model)
        print(sub_index/len(models))
        if(firstChunk):
            submissions.append(submission)
        else:
            submissions[sub_index]=pd.concat([submissions[sub_index],submission])

print("Evaluating test data...")
        
filename='../input/test_numeric.csv'
chunknum = 0
for chunk in pd.read_csv(filename, header=0, chunksize=100000):
    process(chunk, chunknum == 0,models)
    chunknum = chunknum +1
    clear_output()
    print("%d percent" % int((chunknum*100000/1183748)*100))

Evaluating test data...


NameError: name 'models' is not defined

In [ ]:
pickle.dump( submissions, open( "../submissions.p", "wb" ) )

In [ ]:
i=0
column_names=[]
for df in submissions:
    column_name = 'Response_'+str(i)
    column_names.append(column_name)
    df.columns=['Id',column_name]
    i = i+1
    
final_submission = submissions[0]
for i in range(1,len(submissions)):
    final_submission = pd.merge(final_submission,submissions[i], on='Id', how='outer')

final_submission['Response'] = final_submission[column_names].mean(axis=1)
final_submission['Response'] = final_submission['Response'].map(lambda x: int(1) if x>0 else int(0))
final_submission= final_submission.drop(column_names,axis=1)

with open('../submission.csv', 'a') as f:
    final_submission.to_csv(f, header=True, index=False)